**Troubleshooting Scripts - In-Memory OLTP**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

Look at companion materials from "Expert SQL Server In-Memory OLTP" book ([https://aboutsqlserver.com/publications](https://aboutsqlserver.com/publications)) for additional troubleshooting scripts

**Memory Consumption in memory-optimized tables.**

In [1]:
SELECT
    ms.object_id
    ,s.name + '.' + t.name AS [table]
    ,ms.memory_allocated_for_table_kb
    ,ms.memory_used_by_table_kb
    ,ms.memory_allocated_for_indexes_kb
    ,ms.memory_used_by_indexes_kb
FROM
    sys.dm_db_xtp_table_memory_stats ms WITH (NOLOCK)
        LEFT OUTER JOIN sys.tables t WITH (NOLOCK) ON
            ms.object_id = t.object_id
        LEFT OUTER JOIN sys.schemas s WITH (NOLOCK) ON 
            t.schema_id = s.schema_id
ORDER BY
    ms.memory_allocated_for_table_kb DESC;

Warning: The join order has been enforced because a local join hint is used.

(0 rows affected)

Total execution time: 00:00:00.022

object_id,table,memory_allocated_for_table_kb,memory_used_by_table_kb,memory_allocated_for_indexes_kb,memory_used_by_indexes_kb


**Oldest in-memory OLTP transactions**

In [2]:
SELECT TOP 10
    t.session_id
    ,t.transaction_id
    ,t.begin_tsn
    ,t.end_tsn
    ,t.state_desc
    ,t.result_desc
    ,SUBSTRING(
        qt.text
        ,er.statement_start_offset / 2 + 1
        ,(CASE er.statement_end_offset
             WHEN -1 THEN datalength(qt.text)
             ELSE er.statement_end_offset
          END - er.statement_start_offset
        ) / 2 +1
    ) AS SQL
FROM 
    sys.dm_db_xtp_transactions t WITH (NOLOCK)
        LEFT OUTER JOIN sys.dm_exec_requests er ON
            t.session_id = er.session_id
        CROSS APPLY sys.dm_exec_sql_text(er.sql_handle) qt
WHERE
    t.state IN (0,3) /* ACTIVE/VALIDATING */
ORDER BY 
    t.begin_tsn
OPTION (RECOMPILE, MAXDOP 1);

(0 rows affected)

Total execution time: 00:00:00.005

session_id,transaction_id,begin_tsn,end_tsn,state_desc,result_desc,SQL
